In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def extract_data(urls):
    """
    Обходит список страниц Avito и собирает из каждой карточки три поля:
      - name  : заголовок объявления
      - price : цена (как текст, без нормализации)
      - info  : краткое описание/доп. сведения

    Защита от дублей:
      - внутри прохода используем множество `unique_entries`, чтобы не складировать повторяющиеся записи.
      - если на текущей странице не появилось ни одной новой записи (полный дубль) — останавливаем цикл.

    Параметры:
      urls : список URL разделов Avito (можно один)

    Возвращает:
      pandas.DataFrame со столбцами ['name', 'price', 'info'].
    """
    # Списки-аккумуляторы для финального DataFrame
    all_names = []
    all_prices = []
    all_infos = []

    # Глобальный набор уже встреченных записей, чтобы отсекать повторы между страницами
    unique_entries = set()

    # Проходимся по всем переданным страницам
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Находим контейнеры карточек.
        # ВАЖНО: классы у Avito меняются со временем — если карточки не находятся,
        #        проверь верстку и замени селектор ниже на актуальный.
        snippets = soup.find_all("div", class_="iva-item-body-oMJBI")

        # Локальный набор записей, найденных именно на этой странице
        current_entries = set()

        # Достаём из каждой карточки нужные поля
        for snippet in snippets:
            # Внутренние блоки карточки (заголовок/цена/описание).
            # Эти классы тоже могут меняться — подставляй новые при необходимости.
            l_name_element  = snippet.find("div", class_="iva-item-title-KE8A9")
            l_price_element = snippet.find("div", class_="price-priceContent-I4I3p")
            l_info_element  = snippet.find("div", class_="iva-item-bottomBlock-VewGa")

            # Безопасно читаем текст (если блок не найден — ставим заглушку)
            l_name  = l_name_element.get_text(strip=True)  if l_name_element  else "No name"
            l_price = l_price_element.get_text(strip=True) if l_price_element else "No price"
            l_info  = l_info_element.get_text(strip=True)  if l_info_element  else "No info"

            # Запись для дедупликации (кортеж из трёх полей)
            entry = (l_name, l_price, l_info)

            # Если такой записи ещё не было — сохраняем в итоговые списки
            if entry not in unique_entries:
                all_names.append(l_name)
                all_prices.append(l_price)
                all_infos.append(l_info)
                current_entries.add(entry)

        # Обновляем глобальный набор уникальных записей
        unique_entries.update(current_entries)

        # Если текущая страница не дала ни одной новой записи — дальше смысла нет
        if not current_entries:
            print(f"Страница {url} дублирует предыдущие данные, завершаем цикл.")
            break

    # Складываем всё в DataFrame
    df = pd.DataFrame(list(zip(all_names, all_prices, all_infos)),
                      columns=['name', 'price', 'info'])
    return df

In [3]:
# Пример использования (одна категория, одна ссылка):
urls = [
    'https://www.avito.ru/moskva/odezhda_obuv_aksessuary/zhenskaya_odezhda-ASgBAgICAUTeAtYL'
]

# Получение данных
df = extract_data(urls)

# Сохранение в CSV (сырые текстовые поля без нормализации)
filename = 'Avito.csv'
df.to_csv(filename, index=False)

# Быстрый просмотр результата в консоли
df

,name,price,info
0,Брюки женские zara xl,4 500₽,Женские брюки Zara в размере XL. Коричневый цв...
1,Футболка оверсайз хлопок,2 400₽,Универсальная Хлопковая футболка оверсайз со в...
2,"Топ летний, бюстгальтер без косточек H&M",1 000₽,"Топы и бюстгальтера без косточек, с мягкой чаш..."
3,Куртка женская,2 000₽,Женская куртка из болоньевой ткани в черном цв...
4,Пальто Zara,2 987₽,Ищете качественный товар по отличной цене? Тог...
...,...,...,...
75,Рубашка женская базовая,1 200₽,Представляем светло голубую рубашку в полосочк...
76,Водолазка с коротким рукавом 42-58 опт/розница,750₽,Водолазка с коротким рукавом.\n\nМатериал: 70%...
77,Женская куртка Stussy,3 790₽,Куртка Stussy Micro Ripstop.\n\nПодарок в зака...
78,Вещи Н&М и С&А на доставку,220₽,Коробка вещей на доставку.\n\nХочешь покупать ...
